Get community building samples from euss and renormalize the sample weight

In [3]:
import pandas as pd
from pathlib import Path
import numpy as np
import getpass

user = getpass.getuser()
print(f"The user is {user}")

The user is lliu2


In [4]:
df = pd.read_parquet("/Users/lliu2/Documents/GitHub/ResStock/euss_cleap/data_/community_building_samples_with_upgrade_cost_and_bill/duluth/processed_upgrade_results.parquet")
df

,building_id,build_existing_model.sample_weight,build_existing_model.geometry_building_horizontal_location_sfa,build_existing_model.mechanical_ventilation,build_existing_model.door_area,build_existing_model.range_spot_vent_hour,build_existing_model.hvac_shared_efficiencies,build_existing_model.eaves,build_existing_model.hvac_heating_efficiency,build_existing_model.income_recs_2020,...,saving_energy.total_mmbtu,pct_saving_energy.total_%,saving_emission.total_kgCO2e,pct_saving_emission.total_%,saving_bill.total_usd,pct_saving_bill.total_%,saving_cbill.total_usd,pct_saving_cbill.total_%,post-upgrade_energy_burden_2019_bills.%,post-upgrade_energy_burden_2023_cbills.%
0,667,242.131013,None,None,20 ft^2,Hour8,None,2 ft,"Electric Furnace, 100% AFUE",<20000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1295,242.131013,None,None,20 ft^2,Hour3,None,2 ft,"Fuel Wall/Floor Furnace, 68% AFUE",20000-39999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1434,242.131013,None,None,20 ft^2,Hour8,None,2 ft,"Fuel Furnace, 80% AFUE",100000-149999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1602,242.131013,None,None,20 ft^2,Hour16,None,2 ft,Other,60000-99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2451,242.131013,None,None,20 ft^2,Hour16,None,2 ft,"Fuel Furnace, 80% AFUE",20000-39999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,548936,242.131013,None,None,20 ft^2,Hour12,None,2 ft,"Fuel Furnace, 92.5% AFUE",150000+,...,88.989,44.04,8434.457511,66.05,-1805.031833,-82.06,-526.726600,-23.29,1.63,1.13
1213,549167,242.131013,None,None,20 ft^2,Hour10,None,2 ft,"Electric Baseboard, 100% Efficiency",40000-59999,...,114.857,69.87,4262.852173,70.32,3946.489967,68.41,2822.160364,69.69,4.43,2.98
1214,549273,242.131013,None,None,20 ft^2,Hour19,None,2 ft,"Electric Baseboard, 100% Efficiency",150000+,...,112.806,69.30,4717.949645,72.04,3556.872352,65.95,2533.961585,67.18,1.07,0.72
1215,549876,242.131013,None,None,20 ft^2,Hour17,None,2 ft,"Fuel Furnace, 80% AFUE",20000-39999,...,58.266,72.47,4310.379543,83.22,67.643320,7.13,313.821979,36.16,3.96,2.49


In [5]:
## Dwelling unit count by community 
Total_dwelling_unit_count = {
    "duluth": 39762,
} # TODO add others here

In [6]:
community = 'duluth' # TODO change community name

In [7]:
# set path
if user == "ylou2":
    euss_dir = 'data_/euss_res_final_2018_550k_20220901/' # TODO
elif user == "lliu2":
    euss_dir = "/Volumes/Lixi_Liu/euss_aws"
else:
    print(f"new user={user}, set euss_dir path here with an elif statement")

euss_dir = Path(euss_dir).resolve()
output_dir = Path(".").resolve() / "data_" / "community_building_samples" / community

print(f"euss_dir: {euss_dir}")
print(f"output_dir: {output_dir}")

euss_dir: /Volumes/Lixi_Liu/euss_aws
output_dir: /Users/lliu2/Documents/GitHub/ResStock/euss_cleap/data_/community_building_samples/duluth


In [8]:
building_id_weight = pd.read_csv(f'data_/downsampled_buildings_id/euss1_2018_results_up00__downsampled_method1__{community}.csv')
building_id_weight['building_id'] = building_id_weight['building_id'].astype(int)
building_id_weight = building_id_weight.set_index('building_id')
# turn off normalization
# building_id_weight["sample_weight"] *= Total_dwelling_unit_count[community] / building_id_weight["sample_weight"].sum() # normalize 
# assert building_id_weight["sample_weight"].sum() == Total_dwelling_unit_count[community], "renormalization failed"

print(f"building_id_weight for {community} has {len(building_id_weight)} rows")

for i in range(11):
    euss_up = pd.read_parquet(euss_dir / f'results_up{i:02d}.parquet', engine='pyarrow')
    euss_up["building_id"] = euss_up["building_id"].astype(int)
    community_up = euss_up.join(building_id_weight, on="building_id", how="right")
    n_downselected = len(community_up)

    community_up = community_up.loc[community_up["completed_status"] == "Success"].reset_index(drop=True) # drop failed sims
    community_up.to_parquet(output_dir / f"up{i:02d}.parquet")
    
    print(f" - for up{i:02d}, community_up has {n_downselected} rows after downselection, and {len(community_up)} rows after dropping failed sims")

print("all done!")


building_id_weight for duluth has 1217 rows
 - for up00, community_up has 1217 rows after downselection, and 1217 rows after dropping failed sims
 - for up01, community_up has 1217 rows after downselection, and 1048 rows after dropping failed sims
 - for up02, community_up has 1217 rows after downselection, and 1093 rows after dropping failed sims
 - for up03, community_up has 1217 rows after downselection, and 1217 rows after dropping failed sims
 - for up04, community_up has 1217 rows after downselection, and 1217 rows after dropping failed sims
 - for up05, community_up has 1217 rows after downselection, and 1159 rows after dropping failed sims
 - for up06, community_up has 1217 rows after downselection, and 1214 rows after dropping failed sims
 - for up07, community_up has 1217 rows after downselection, and 1037 rows after dropping failed sims
 - for up08, community_up has 1217 rows after downselection, and 1217 rows after dropping failed sims
 - for up09, community_up has 1217 row

### Get summary

In [81]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)


In [86]:
metric = 'saving_energy.total_mmbtu'
dfgb = df.groupby(["upgrade_name"])
pd.concat([
    dfgb["sample_weight"].count().rename("n_samples"),
    dfgb["sample_weight"].sum().rename("n_represented"),
    dfgb.apply(lambda x: weighted_mean(x, metric)),
    dfgb.apply(lambda x: weighted_percentile(x, metric, percents=0.25)),
    dfgb.apply(lambda x: weighted_percentile(x, metric, percents=0.75)),
], axis=1)

,n_samples,n_represented,saving_energy.total_mmbtu wt_mean,saving_energy.total_mmbtu wt_p0.25,saving_energy.total_mmbtu wt_p0.75
upgrade_name,,,,,
Basic Enclosure,1048,34612.492977,25.788223,6.467994,34.395091
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,91.878534,42.409306,121.522125
Basic Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,93.092600,43.622250,123.698558
Electric Clothes Dryer,176,5914.743706,0.473697,0.260000,0.600292
Electric Cooking,385,12722.230006,1.246375,1.190000,1.661616
Enhanced Enclosure,1093,35800.509360,33.426529,9.256700,46.731490
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,93.194065,43.373831,123.105097
Enhanced Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,94.408130,44.146708,125.047591
Heat Pump Clothes Dryer,1076,32436.993494,0.868651,0.476000,1.178000


In [92]:
metric = 'saving_cbill.total_usd'
dfgb = df.groupby(["upgrade_name"])
pd.concat([
    dfgb["sample_weight"].count().rename("n_samples"),
    dfgb["sample_weight"].sum().rename("n_represented"),
    dfgb.apply(lambda x: weighted_mean(x, metric)),
    dfgb.apply(lambda x: weighted_percentile(x, metric, percents=0.25)),
    dfgb.apply(lambda x: weighted_percentile(x, metric, percents=0.75)),
], axis=1)

,n_samples,n_represented,saving_cbill.total_usd wt_mean,saving_cbill.total_usd wt_p0.25,saving_cbill.total_usd wt_p0.75
upgrade_name,,,,,
Basic Enclosure,1048,34612.492977,240.344864,58.168620,329.469548
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,350.315223,15.130565,738.888071
Basic Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,366.229714,41.926153,757.575379
Electric Clothes Dryer,176,5914.743706,-19.717749,-25.248421,-10.646974
Electric Cooking,385,12722.230006,-6.574066,-6.700608,-4.633062
Enhanced Enclosure,1093,35800.509360,314.871643,73.034127,447.545619
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1217,39762.000000,382.639206,36.540474,771.424427
Enhanced Enclosure + High Efficiency Whole Home Electrification,1217,39762.000000,398.553697,60.235657,785.442050
Heat Pump Clothes Dryer,1076,32436.993494,15.625887,0.684497,24.595650
